# 2.5.7: Glucose (A new tool for initial value problems)


### Warning solve_ivp can't deal with the insulin data as discrete data I[t]: it needs a function so it can define it at anytime step I(t).   Should I just do all of the simulations with change functions?

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In this notebook, we'll implement the minimal model in a new way, by using a SciPy function called `solve_ivp` to solve the differential equation using a better algorithm.  We'll see that `solve_ivp` is faster and more accurate than `run_simulation`.
As a result, we will use it for the models in the rest of the class.

<br>

First, make a system and state by running this cell:

In [4]:
import pandas as pd
import numpy as np

filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/glucose_insulin.xlsx'
data = pd.read_excel(filename, index_col=0)
data.columns = ['glucose', 'insulin']

# Define parameters
G0 = 270
X0 = 0
k1 = 0.02
k2 = 0.02
k3 = 1.5e-05

# Define simulation parameters based on data
t_0 = data.index[0]
t_end = data.index[-1]
Gb = data.glucose[t_0]
Ib = data.insulin[t_0]
dt = (t_end-t_0)/(data.glucose.size -1)

# The system needs an initial state object called 'init' because we'll
# use that later
state = pd.Series(dict(G=G0, X=X0),dtype=np.float64)
system = dict(init=state, k1=k1,k2=k2,
                  k3=k3,dt=dt, Gb=Gb, Ib=Ib, I=data.insulin,
                  t_0=t_0, t_end=t_end)

In [2]:
from urllib.request import urlretrieve

location = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/'
folder = 'Support_files/'
name = 'ModSimPy.ipynb'
local, _ = urlretrieve(location + folder + name, name)
%run /content/$name

## Using a Solver to Solve Differential Equations

In the previous notebook, we rewrote the differential equations as difference equations with a finite time step, `dt`.
When $dt$ is very small, or more precisely *infinitesimal*, the difference equations are the same as the differential equations.
But in our simulations, $dt$ is 2 min, which is not very small, and definitely not infinitesimal.

<br>

In effect, the simulations assume that the derivatives $dG/dt$ and $dX/dt$ are constant during each 2 min time step.
This method, evaluating derivatives at discrete time steps and assuming that they are constant in between, is called *Euler's method*.  Euler's method is good enough for many problems, but sometimes it is not very accurate.
In that case, we can usually make it more accurate by decreasing the size of `dt`.
But then it is not very efficient.

<br>

There are other methods that are more accurate and more efficient than Euler's method.
SciPy provides several of them wrapped in a function called `solve_ivp`.
The `ivp` stands for *initial value problem*, which is the term for problems like the ones we've been solving, where we are given the initial conditions and try to predict what will happen.

### Introducing a Slope Function

The ModSim library provides a function called `run_solve_ivp` that makes `solve_ivp` a little easier to use.  To use it, we have to provide a *slope function*, which is similar to an change function, except that it uses the terminology of differential equations.  The slope function tells us how much the value of variable is changing at an instant in time.  It takes the same parameters as a change function, too: a time stamp, a state object, and a system object.

<br>

Compare the `change_func` we wrote earlier to our new `slope_func`, which evaluates the differential equations of the minimal model.

In [10]:
def change_func(t, state, system):
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']

    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I[t] - Ib) - k2 * X

    state.G += dGdt * dt
    state.X += dXdt * dt

    return state

def slope_func(t, state, system):
    G, X = state
    k1, k2, k3, dt = system['k1'],system['k2'], system['k3'], system['dt']
    I, Ib, Gb = system['I'], system['Ib'], system['Gb']

    dGdt = -k1 * (G - Gb) - X*G
    dXdt = k3 * (I[t] - Ib) - k2 * X

    return dGdt, dXdt

`slope_func` is a little simpler than `change_func` because it only computes the rates of change 'dGdt' and 'dXdt' (that is, it computes the slopes). It doesn't update the state variables directly: `solve_ivp` does that for us.  

<br>**Notice that the slope terms that it returns are in the same order as the state variables (i.e G and dGdt both come first, X and dXdt come second).**  This order is important: it is the way that `solve_ivp` keeps track of the changes to the state variables.

### Using the Solver

Now we can call `run_solve_ivp` like this:

In [11]:

t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
results, details = run_solve_ivp(system, slope_func,t_eval=t_array)

Notice how we call `run_solve_ivp`:

<br>

* Like `run_simulation`, it takes a `System` object and a slope function as parameters.

* The `t_eval` option tells the solver at which time points to evaluate a solution: this is optional and is called a "keyword argument", because you need to set the argument equal to the keyword `t_eval` if you want to use it.

* It returns two values: a `DataFrame` of results, which we assign to `results2`, and an `OdeResult` object, which we assign to `details`.

<br>

The `OdeResult` object contains information about how the solver ran, including a success code and a diagnostic message.  Make sure that you include two variable names on the left hand side when you run `run_solve_ivp`!

In [ ]:
details.success

In [ ]:
details.message

It's important to check these messages after running the solver, in case anything went wrong.

<br>

The `DataFrame` has one row for each time step and one column for each state variable. In this example, the rows are time from 0 to 182 minutes; the columns are the state variables, `G` and `X`.  Here are the first few time steps:

In [ ]:
results.head()

And here's a plot of $G$:

In [ ]:
results.G.plot(style='-', label='solve ivp',xlabel='Time (min)',
         ylabel='Concentration (mg/dL)',legend=True);

## Comparing `solve_ivp` to `run_simulation`

How much more accurate are our results when we use `solve_ivp` rather than `run_simulation`?  Let's compare the two.

<br> First, we'll need to get the results using our old methods.

In [ ]:
def run_simulation(system, state, change_func):
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)

    frame = pd.DataFrame(index=t_array,
                      columns=state.index,
                        dtype=np.float64)
    frame.iloc[0] = state

    for i in range(n-1):
        t = t_array[i]
        frame.iloc[i+1] = change_func(t, state, system)

    return frame

system, state = make_system(G0, k1, k2, k3, dt, data)
results_rs = run_simulation(system, state,change_func)
system, state = make_system(G0, k1, k2, k3, dt, data)
results_ivp, details = run_solve_ivp(system, slope_func,t_eval=results_rs.index)

Because we used `t_eval=result_rs.index`, the time stamps in `results_ivp` are the same as in `results_rs`, which makes them easier to compare.  The following figure shows the results from `run_solve_ivp` along with the results from `run_simulation`:

In [ ]:
results_rs.G.plot(style='--', label='run_simulation',legend=True)
results_ivp.G.plot(style='-', label='solve ivp',xlabel='Time (min)',
         ylabel='Concentration (mg/dL)',legend=True);

The differences are barely visible.
We can compute the relative differences like this:



In [ ]:
diff = results_rs.G - results_ivp.G
percent_diff = diff / results_ivp.G * 100

And we can use `describe` to compute summary statistics:

In [ ]:
percent_diff.abs().describe()


The mean relative difference is about 0.65% and the maximum is a little more than 1%.
Here are the results for `X`.

In [ ]:
results_rs.X.plot(style='--', label='run_simulation',legend=True)
results_ivp.X.plot(style='-', label='solve ivp',xlabel='Time (min)',
         ylabel='Concentration (arbitrary units)',legend=True);

These differences are little bigger, especially at the beginning.  If we use `run_simulation` with smaller time steps, the results are more accurate, but they take longer to compute.  For some problems, we can find a value of `dt` that produces accurate results in a reasonable time. However, if `dt` is *too* small, the results can be inaccurate again. So it can be tricky to get it right.

In [ ]:
diff = results.X - results2.X
percent_diff = diff / (results2.X.mean()) * 100
percent_diff.abs().describe()

The advantage of `run_solve_ivp` is that it chooses the step size automatically in order to balance accuracy and efficiency.  You can use keyword arguments to adjust this balance, but most of the time the results are accurate enough, and the computation is fast enough, without any intervention.

### Exercise 1

Our solution to the differential equations is only approximate because we used a finite step size, `dt=2` minutes.
If we make the step size smaller, we expect the solution to be more accurate.  Run the simulation with `dt=1` and compare the results for `G` with the `dt = 2` results.

In [ ]:
# Run simulation with dt = 1 and dt = 2
# Save the results as results1 and results2




In [ ]:
#Plot both results with the glucose data
data.glucose.plot(style='o', alpha=0.5, label='glucose data',legend=True)
results1.G.plot(style='--', color='C1', label='dt = 1',legend=True)
results2.G.plot(style='-', color='C3', label='dt = 2',
                xlabel='Time (min)',
                ylabel='Concentration (mg/dL)',
                ylim=[80,270],legend=True);

### Exercise 2

Comment on the results of your comparison.  Do the results differ significantly between a dt = 1 and dt = 2?   Now try larger dt values: do we still get usable results with a dt = 5? 10? 20?

<br>

Why might it be useful to have a larger dt even if it costs us a little accuracy?

✅ ✅  Answer here.